In [5]:
import pandas as pd
import xgboost
import numpy as np
import plotly as plt

In [6]:
import datetime
import requests
import math
import pandas as pd
from calendar import monthrange

def get_td_bearer_token() -> str:
    """ Retrieves a bearer token from Team Dynamix to get access.     
    Returns    
    -------    
    str        
    a string containing the bearer token        
    Example: "Bearer: HsJKsdjj2iU8..."    
    """    
    username = "automationrobot@byui.edu"    
    password = "Robo493!"    
    auth_url = "https://td.byui.edu/TDWebApi/api/auth"    
    data = {"username": username, "password": password}
    response = requests.post(auth_url, data)
    bearer_token = "Bearer " + response.text    
    
    return bearer_token

In [7]:
def get_td_report(report_id : int, renaming = {}) -> pd.DataFrame:
        
    url = f'https://td.byui.edu/TDWebApi/api/reports/{report_id}?withData={True}&dataSortExpression='    
    bearer_token = get_td_bearer_token()
    authorization = {"Authorization": bearer_token}
    response = requests.get(url, headers=authorization)
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if response.status_code == 401:
           print("Warning: Repeated attempts to access TD will cause the account to lock.")
        raise    
    report = format_attributes(pd.DataFrame(dict(response.json()).get("DataRows")), renaming)
    return report

In [8]:
def format_attributes(report : pd.DataFrame, renaming : dict):
    """ Gives names to the otherwise unknown attribute IDs on reports.         
    Parameters    
    ----------    
    report : pd.DataFrame        
    Any TeamDynamix report.    
    Returns    -------    
    pd.DataFrame        
    the same TeamDynamix report with labeled column headers
    """    
    report = report.rename(renaming, axis=1)
    return report

In [9]:
def get_date_instance(year, month, day_index, instance_number):
    assert int(month), "'month' has to be 'int' type"    
    assert int(day_index), "'month' has to be 'int' type"    
    assert int(instance_number), "'instance_number' has to be 'int' type"    
    curr_instance = 0    
    days_in_month = monthrange(year, month)[1]
    for day in range(1, days_in_month + 1):
        if curr_instance == instance_number:
            break        
        else:
            date = datetime.datetime(year, month, day)    
            if date.weekday() == day_index:
                curr_instance += 1    
    return date

In [10]:
def get_MT_datetime(row):
    """    row (string) : needs to be on format yyyy-mm-ddTHH:MM:SS    
    """    
    date = datetime.datetime.strptime(row[:16], "%Y-%m-%dT%H:%M")

    mdt_start = get_date_instance(date.year, 3, 6, 2) # second Sunday of March    
    mdt_end = get_date_instance(date.year, 11, 6, 1)  # first Sunday of November

    #if MDT subtract 6    
    if mdt_start <= date < mdt_end:        
        return date - datetime.timedelta(hours=6)
    # else MST subtract 7    
    else:
        return date - datetime.timedelta(hours=7)

In [13]:
rename_map = {
            '14231':'office_list'            
            }

df = get_td_report(21883, rename_map)
df['date'] = df.apply(lambda row : get_MT_datetime(row['CreatedDate']), axis=1)
df.head()

HTTPError: 401 Client Error: Unauthorized for url: https://td.byui.edu/TDWebApi/api/reports/21883?withData=True&dataSortExpression=